**install libraries**


In [ ]:
!pip install -q langgraph google-generativeai python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.5/152.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 15.6 MB/s eta 0:00:00


**Gemini models**

**set up the gemini API**

In [ ]:
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash-latest')

**define langgraph nodes (CAREER COUNSELLING FLOW)**

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
from uuid import uuid4

class AgentState(TypedDict):
  user_input: str
  interests: str
  personality: str
  careers: str
  resources: str

**Create  node**

In [ ]:
def extract_interests(state):
  prompt = f"from this input , extract the user's interests or traits: {state['user_input']}"
  print(f"Prompt for extract_interests: {prompt}")
  response = model.generate_content(prompt)
  print(f"Response from extract_interests: {response}")
  if response is not None and hasattr(response, 'text'):
    state['interests'] = response.text
  else:
    state['interests'] = "Could not extract interests. Please try again."
  return state

def analyze_personality(state):
  if state['interests'] == "Could not extract interests. Please try again.":
    state['personality'] = "Cannot analyze personality due to previous error."
    return state

  prompt = f"based on these interest , describe the personality traits: {state['interests']}"
  print(f"Prompt for analyze_personality: {prompt}")
  response = model.generate_content(prompt)
  print(f"Response from analyze_personality: {response}")
  if response is not None and hasattr(response, 'text'):
    state['personality'] = response.text
  else:
    state['personality'] = "Could not analyze personality."
  return state

def suggest_careers(state):
  if state['personality'] == "Could not analyze personality." or \
     state['personality'] == "Cannot analyze personality due to previous error.":
    state['careers'] = "Cannot suggest careers due to previous error."
    return state

  prompt = f"suggest 3 career options for someone with this traits:  {state['personality']}"
  print(f"Prompt for suggest_careers: {prompt}")
  response = model.generate_content(prompt)
  print(f"Response from suggest_careers: {response}")
  if response is not None and hasattr(response, 'text'):
    state['careers'] = response.text
  else:
    state['careers'] = "Could not suggest careers."
  return state

def reommend_resources(state):
  if state['careers'] == "Could not suggest careers." or \
     state['careers'] == "Cannot recommend resources due to previous error.":
    state['resources'] = "Cannot recommend resources due to previous error."
    return state

  prompt = f"recommend learning resources(courses/videos) for:  {state['careers']}"
  print(f"Prompt for reommend_resources: {prompt}")
  response = model.generate_content(prompt)
  print(f"Response from reommend_resources: {response}")
  if response is not None and hasattr(response, 'text'):
    state['resources'] = response.text
  else:
    state['resources'] = "Could not recommend resources."
  return state

**conect nodes using langgraph**

In [ ]:
graph_builder = StateGraph(AgentState)

graph_builder.add_node("extract interests", extract_interests)
graph_builder.add_node("analyze personality", analyze_personality)
graph_builder.add_node("suggest careers", suggest_careers)
graph_builder.add_node("recommend resources", reommend_resources)

graph_builder.set_entry_point("extract interests")
graph_builder.add_edge("extract interests", "analyze personality")
graph_builder.add_edge("analyze personality", "suggest careers")
graph_builder.add_edge("suggest careers", "recommend resources")
graph_builder.set_finish_point("recommend resources")

graph = graph_builder.compile()

**Run your agent**

In [ ]:
user_text = input("tell me about your interests an strengths: ")
user_input_data ={
    "user_input": user_text,
    "interests": "",
    "personality" : "",
    "careers": "",
    "resources": ""
}

# Make sure to run the cell above that defines 'graph' before running this cell.
final_state = None
for output in graph.stream(user_input_data):
    final_state = output # Keep track of the last output

if final_state and final_state.get('interests') == "Could not extract interests. Please try again.":
    print("\nCareer counselling could not be completed based on your input. Please try again with a different description of your interests and strengths.")
elif final_state:
    print("\n--- Career Counselling Results ---")
    print(f"\nINTERESTS:\n{final_state.get('interests', 'N/A')}")
    print(f"\nPERSONALITY:\n{final_state.get('personality', 'N/A')}")
    print(f"\nCAREERS:\n{final_state.get('careers', 'N/A')}")
    print(f"\nRESOURCES:\n{final_state.get('resources', 'N/A')}")
else:
    print("\nAn unknown error occurred during the career counselling process.")

tell me about your interests an strengths: art and craft
Prompt for extract_interests: from this input , extract the user's interests or traits: art and craft
Response from extract_interests: response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "The user's interest is in **art and craft**.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.2927338160001315
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 16,
        "candidates_token_count": 13,
        "total_token_count": 29
      },
      "model_version": "gemini-1.5-flash-latest"
    }),
)
Prompt for analyze_personality: based on these interest , describe the personality traits: The user's interest is in **art and craft**.

Response from analyze

ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash-latest:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 3511.64ms


Response from suggest_careers: response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Based on the listed traits, here are three career options that would be a good fit for someone with an interest in art and craft:\n\n1. **Jewelry Designer/Maker:** This career perfectly blends creativity, imagination, dexterity, detail-orientation, and aesthetic appreciation.  The ability to problem-solve is crucial for overcoming technical challenges in design and construction.  Patience is essential for the meticulous work involved, and self-expression is readily channeled through unique designs.  Resilience is helpful for navigating the competitive aspects of the market.\n\n2. **Graphic Designer:**  While not strictly a \"craft,\" graphic design relies heavily on creativity, imagination, aesthetic appreciation, and problem-s

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemin